In [37]:
from datetime import datetime, date, time, timedelta
from itertools import islice
from geopy.distance import great_circle
from operator import itemgetter
import matplotlib.pylab as plt
from pandas import DataFrame
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy
import json
import math


DELTA_GNSS = 0.05 #equivalent to about 5-6 km
DELTA_GNSS_INIT = 1 #equivalent to about 5-6 km
DELTA_POS = 0.0001 #equivalent to about 100 m

#six decimal are enough for a decimeter precision
#0.1 = 11.1 km
#0.01 = 1.11 km
#...
#0.000001 = 0.111 m
uav_latitude = 45.0059289
uav_longitude = 10.461025
uav_coo = (uav_latitude, uav_longitude)

#support variables
bts_latitude = 0
bts_longitude = 0

#Load BTS database
BTS_DATA = pd.read_csv('bts_windtre.csv', sep=';')
BTS_DATA = BTS_DATA.drop(columns=['tech','mcc','mnc','lac_tac','cid','psc_pci','arfcn','azimuth','height','tilt_mech','tilt_el'])
BTS_DATA = BTS_DATA.round(5)

BTS_DATA = BTS_DATA.drop_duplicates(subset=['site_name'])
BTS_DATA = BTS_DATA.drop_duplicates(subset=['cell_lat'])
BTS_DATA = BTS_DATA.drop_duplicates(subset=['cell_long'])

###In order to limit processing, limit the DF to cells in 100km radius
Nearby_Cells_Lat = abs(BTS_DATA['cell_lat']-uav_coo[0])<=DELTA_GNSS_INIT
Nearby_Cells_Lon = abs(BTS_DATA['cell_long']-uav_coo[1])<=DELTA_GNSS_INIT

BTS_DATA = BTS_DATA[Nearby_Cells_Lon]
BTS_DATA = BTS_DATA[Nearby_Cells_Lat]
###Must be deleted in the final version

<ipython-input-37-8d9d0b6f8fe6>:47: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [38]:
#Bearing Angle
def calculate_bearing(pointA, pointB):
    """
    Calculates the bearing between two points.
    The formulae used is the following:
        θ = atan2(sin(Δlong).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
    :Parameters:
      - `pointA: The tuple representing the latitude/longitude for the
        first point. Latitude and longitude must be in decimal degrees
      - `pointB: The tuple representing the latitude/longitude for the
        second point. Latitude and longitude must be in decimal degrees
    :Returns:
      The bearing in degrees
    :Returns Type:
      float
    """
    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

#Find nearby cells
def CellFind(uav_coo):

    Nearby_Cells_Lat = abs(BTS_DATA['cell_lat']-uav_coo[0])<=DELTA_GNSS
    Nearby_Cells_Lon = abs(BTS_DATA['cell_long']-uav_coo[1])<=DELTA_GNSS

    NEARBY_BTS = BTS_DATA[Nearby_Cells_Lon]
    NEARBY_BTS = NEARBY_BTS[Nearby_Cells_Lat]

    #Compute distance and bearing of nearby cells
    nearby_bts_distances = []
    nearby_bts_bearing = []

    for i in range(len(NEARBY_BTS)):
        cell = NEARBY_BTS.iloc[i]
        bts_coo = (cell.cell_lat, cell.cell_long)
        nearby_bts_distances.append(great_circle(uav_coo, bts_coo).m)
        nearby_bts_bearing.append(calculate_bearing(uav_coo, bts_coo))


    NEARBY_BTS['distance'] = nearby_bts_distances
    NEARBY_BTS['bearing'] = nearby_bts_bearing

    NEARBY_BTS = NEARBY_BTS.sort_values(by='distance')

    return(NEARBY_BTS)

In [39]:
#Import UAV Log network Ping data from JSON file
data = []
with open('UAV_LOG.json') as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except:
            print('Skip line')

#Convert JSON data from Python dict to DF
UAV_LOG = pd.json_normalize(data)

#Drop unusfull data
UAV_LOG = UAV_LOG.drop(columns=['uav_param.status','uav_param.mode','interface','gps_param.groundspeed','gps_param.velocity.vx','uav_param.attitude.pitch','uav_param.attitude.yaw','uav_param.attitude.roll','gps_param.velocity.vy','gps_param.velocity.vz','gps_param.hdop','gps_param.vdop','gps_param.fix_type'])

#Remove duplicate entries
UAV_LOG = UAV_LOG.drop_duplicates('timestamp')
UAV_LOG = UAV_LOG.apply(pd.to_numeric, errors='ignore')

#Remove false flights
UAV_bench = UAV_LOG['uav_param.battery']>=10.0
UAV_LOG = UAV_LOG[UAV_bench]

#Remove no GPS fix data
GPS_Null = UAV_LOG['gps_param.satellites']>=6
UAV_LOG = UAV_LOG[GPS_Null]

#Sort by time for time clustering
UAV_LOG = UAV_LOG.sort_values(by='timestamp')


Skip line
Skip line
Skip line
Skip line
Skip line
Skip line
Skip line
Skip line


In [40]:
#Time Clustering Function
#Convert timestamp of dataframe to python datetime
UAV_LOG.timestamp = pd.to_datetime(UAV_LOG["timestamp"])
UAV_LOG.timestamp = UAV_LOG.timestamp.dt.to_pydatetime()

#CLUSTER LTE DF
#Define the minimum time difference between flights, in order to identy clusters of data
delta = timedelta(minutes=2)
final = list()
tmp = list()

#First row
tmp.append(UAV_LOG.iloc[0,:])

#First datetime
previous = UAV_LOG.iloc[0,0]

#Scan UAV Log dataframe looking for clusters and then append them into a list of dataframes
for index, row in islice(UAV_LOG.iterrows(), 0, None):
    dt = row.timestamp
    if dt - previous > delta:
        final.append(tmp)
        tmp = list()
    tmp.append(row)
    previous = dt

#Append last cluster to the list of DF
final.append(tmp)

#Remove flight with less than 100 rows
flights = []
for i in range(len(final)):
    if len(final[i])>1000:
        flights.append(final[i])

#Print N° of flights
print("Found ", len(flights), " flights with data")

#Print flight lenght
for i in range(len(flights)):
    print("Flight N°", i+1, "has ", len(flights[i]), " rows of data")

print('')

Found  11  flights with data
Flight N° 1 has  1029  rows of data
Flight N° 2 has  1807  rows of data
Flight N° 3 has  2755  rows of data
Flight N° 4 has  2013  rows of data
Flight N° 5 has  1880  rows of data
Flight N° 6 has  5348  rows of data
Flight N° 7 has  6511  rows of data
Flight N° 8 has  5091  rows of data
Flight N° 9 has  4600  rows of data
Flight N° 10 has  4760  rows of data
Flight N° 11 has  4363  rows of data



In [68]:
DF = DataFrame (flights[6])

for log in range(len(DF)):
    uav_coo = (DF.iloc[log]['gps_param.latitude'], DF.iloc[log]['gps_param.longitude'])
    BTS = CellFind(uav_coo)
    #print(BTS.iloc[0])
    #print(DF.iloc[log])


<ipython-input-38-8374e3e56ef3>:47: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [73]:
DATA = []

for f in range(len(DF)):
    coo = (DF.iloc[f]['gps_param.latitude'], DF.iloc[f]['gps_param.longitude'])
    DATA.append(coo)

[(45.0059276, 10.4610245),
 (45.0059276, 10.4610245),
 (45.0059276, 10.4610245),
 (45.0059276, 10.4610245),
 (45.0059276, 10.4610245),
 (45.0059281, 10.4610247),
 (45.0059281, 10.4610247),
 (45.0059281, 10.4610247),
 (45.0059281, 10.4610247),
 (45.0059281, 10.4610247),
 (45.0059283, 10.4610248),
 (45.0059283, 10.4610248),
 (45.0059283, 10.4610248),
 (45.0059283, 10.4610248),
 (45.0059283, 10.4610248),
 (45.0059289, 10.4610246),
 (45.0059289, 10.4610246),
 (45.0059289, 10.4610246),
 (45.0059289, 10.4610246),
 (45.0059289, 10.4610246),
 (45.0059295, 10.4610245),
 (45.0059295, 10.4610245),
 (45.0059295, 10.4610245),
 (45.0059295, 10.4610245),
 (45.0059295, 10.4610245),
 (45.0059299, 10.4610244),
 (45.0059299, 10.4610244),
 (45.0059299, 10.4610244),
 (45.0059299, 10.4610244),
 (45.0059299, 10.4610244),
 (45.0059303, 10.4610242),
 (45.0059303, 10.4610242),
 (45.0059303, 10.4610242),
 (45.0059303, 10.4610242),
 (45.0059303, 10.4610242),
 (45.0059302, 10.4610241),
 (45.0059302, 10.4610241),
 

In [72]:
coo

(45.0059312, 10.461046)

In [43]:
from ipyleaflet import Map, basemaps, Marker, Icon

zoom = 18

m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=uav_coo, zoom=zoom)
icon_drone = Icon(icon_url='https://dash-leaflet.herokuapp.com/assets/icon_plane.png', icon_anchor=[16, 16], icon_size=[21, 21], rotate=True)
icon_bts = Icon(icon_url='http://simpleicon.com/wp-content/uploads/antenna-2-64x64.png', icon_anchor=[32, 32], icon_size=[32, 32])

marker_drone = Marker(location=uav_coo, icon=icon_drone, draggable=False, rotation_angle=55) 
marker_bts = Marker(location=(BTS.iloc[0].cell_lat, BTS.iloc[0].cell_long), icon=icon_bts, draggable=False) 
m.add_layer(marker_bts)
m.add_layer(marker_drone)

DF = DataFrame (flights[6])

#for log in range(len(DF)):
#    m.clear_layers
#    marker_drone = Marker(location=(DF.iloc[log]['gps_param.latitude'], DF.iloc[log]['gps_param.longitude']), icon=icon_drone, draggable=False) 

m

Map(center=[45.0059312, 10.461046], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

In [74]:
import dash_leaflet as dl
from dash import Dash, html

# Rotated custom marker.
iconUrl = "https://dash-leaflet.herokuapp.com/assets/icon_plane.png"
marker = dict(rotate=True, markerOptions=dict(icon=dict(iconUrl=iconUrl, iconAnchor=[16, 16])))

patterns = [dict(repeat='10', dash=dict(pixelSize=5, pathOptions=dict(color='#000', weight=1, opacity=0.2))), dict(offset='16%', repeat='33%', marker=marker)]

rotated_markers = dl.PolylineDecorator(positions=DATA, patterns=patterns)

# Create app.
app = Dash()
app.layout = html.Div(dl.Map([dl.TileLayer(), rotated_markers], zoom=12, center=uav_coo), style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Feb/2022 10:39:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2022 10:39:50] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2022 10:39:50] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2022 10:39:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2022 10:39:53] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2022 10:39:53] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2022 10:39:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2022 10:39:53] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2022 10:39:53] "GET /_dash-dependencies HTTP/1.1" 200 -
